### generate.ipynb
This notebook is for testing the effect of text generation.

In [ ]:
import torch
from torch import nn
import numpy as np
from lgg_model import *
from gensim.models import Word2Vec
# from translate import *

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
lgg_model_path = input("请输入想使用的语言模型(无需添加后缀)：")
lgg_model_path = 'lgg_model_paths/' + lgg_model_path
lgg_model = torch.load(lgg_model_path, map_location='cpu')
lgg_model.eval()

In [ ]:
word_model_path = input("请使用想使用的词汇库(无需添加后缀)：")
word_model_path = 'word_model_paths/' + word_model_path
word_model = Word2Vec.load(word_model_path)
wv = word_model.wv

In [ ]:
from transformers import BertTokenizer, GPT2LMHeadModel
tokenizer = BertTokenizer.from_pretrained("uer/gpt2-chinese-cluecorpussmall")

In [ ]:
lgg_model(torch.tensor([101, 10000, 22, 102]))['logits']

In [ ]:
tokenizer.vocab_size

In [ ]:
words = input("请输入初始文本：")
del_lst = []
lst = list(words)

for i in lst:
    if i not in wv.key_to_index:
        del_lst.append(i)
for i in del_lst:
    lst.remove(i)

data = np.array([])
for i in lst:
    data = np.append(data, wv.key_to_index[i])

count = int(input("请输入想要生成的字(词)数："))

for i in lst:
    print(i, end='')

for i in range(count):
    data = np.stack((data,))
    x = torch.Tensor(data)
    x = x.to(torch.long)
    y = lgg_model(x)[0][-1]
    p = y.detach().numpy()
    p = softmax(p)

    idx = np.random.choice(np.arange(len(wv)), p=p)
    new_word = wv.index_to_key[idx]
    print(new_word, end='')

    lst.append(new_word)
    data = np.append(data, idx)

print('\nGeneration finished.')

In [ ]:
"""GPT-2 model generation"""
"""没用！净给我报错！"""
words = input("请输入初始文本：")
del_lst = []
lst = list(words)

data = []
data.append(101)
for i in lst:
    data.append(tokenizer.encode(i)[1])
data.append(102)
data = torch.tensor(data)
print(data)
print(lgg_model(data)[0][-1])

count = int(input("请输入想要生成的字(词)数："))

for i in lst:
    print(i, end='')

for i in range(count):
    y = lgg_model(data)[0][-1]
    p = y.detach()
    p = torch.softmax(p, dim=0)

    idx = np.random.choice(tokenizer.vocab_size, p=np.array(p))
    new_word = tokenizer.decode(idx)
    print(new_word, end='')

    lst.append(new_word)
    data = torch.hstack((data, torch.tensor([idx])))

print('\nGeneration finished.')

In [ ]:
# translate the generated text into classical Chinese
# to_be_translated = str(lst)
# translated = inference(to_be_translated)
# for i in translated[0]:
#     print(i, end='')